In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time, os
  
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


geckodriver = "~/Downloads/geckodriver-v0.30.0-linux64/geckodriver" # path to the chromedriver executable
os.environ["webdriver.gecko.driver"] = geckodriver

In [10]:
car_cat_lst = ['sports-utilities','pickup-trucks','crossovers','sedans','coupes','convertibles','luxury']
url = 'https://www.carmax.com/cars/pickup-trucks'

In [11]:
driver = webdriver.Firefox()

In [14]:
driver.get(url)

In [15]:
see_more_button = driver.find_element_by_xpath('//div[contains(@id, "see-more")]/div[contains(@class, "see-more")]/a[contains(@class, "kmx-button")]')
driver.execute_script("arguments[0].scrollIntoView(true);", see_more_button)

for i in range(60):
    try:
        see_more_button.click()
        time.sleep(6)
    except:
        continue

In [17]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
df = pd.DataFrame()
cat_lst = ['year-make','model-trim','price','miles']

In [18]:
i = 0
for car in soup.find_all('article', class_ = 'car-tile'):
    car_dict = {}
    
    for key in cat_lst:
        cat = car.find('span', class_ = key).get('class')[0]
        val = car.find('span', class_ = key).text
        car_dict[cat] = val
     

    if i == 0:
        spfc_car_soup = soup.find('article', class_ = 'car-tile')
        spfc_car = spfc_car_soup.find('a').get('href')
    else:
        spfc_car_soup = spfc_car_soup.find_next('article', class_ = 'car-tile')
        spfc_car = spfc_car_soup.find('a').get('href')
    car_url = 'https://www.carmax.com' + spfc_car
    
    driver_car = webdriver.Firefox()
    time.sleep(3)
    try:
        driver_car.get(car_url)
    except:
        driver_car.close()
        print(f'The car, {car_url}, failed to load properly on loop iteration {i}; skipping to next iteration.')
        continue
    
    feat_spec = driver_car.find_element_by_xpath('//div[contains(@class, "information-drawers")]/div[contains(@class, "drawer")]/div[contains(@class, "drawer-header")]/span[contains(@class, "svg-icon-wrapper")]')
    driver_car.execute_script("arguments[0].scrollIntoView(true);", feat_spec)
    
    feat_spec.click()
    
    time.sleep(10)
    
    soup_car = BeautifulSoup(driver_car.page_source, 'html.parser')
    specs = soup_car.find('div', id = "specifications-overview")
    
    spec_dict = {}
    
    ##skipping current iteration if for whatever reason the specific car url doesn't load properly
    try:
        specs.find_all('div', class_ = "specification")
    except:
        driver_car.close()
        print(f'The car, {car_url}, failed to load properly on loop iteration {i}; skipping to next iteration.')
        continue
            
    for spec in specs.find_all('div', class_ = "specification"):
        ps = spec.find_all('p')
        spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text
    if len(soup_car.find_all('div', class_ = "specifications-overview")) > 1:
        perf_specs = soup_car.find_all('div', class_ = "specifications-overview")[1]
        for spec in perf_specs.find_all('div', class_ = "specification"):
            ps = spec.find_all('p')
            spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text
    
        meas_specs = soup_car.find_all('div', class_ = "specifications-overview")[2]
        for spec in meas_specs.find_all('div', class_ = "specification"):
            ps = spec.find_all('p')
            spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text
        
    car_dict.update(spec_dict)
    driver_car.close()
    df = df.append(car_dict, ignore_index = True)
    
    if i % 10 == 0:
        print(i)
    
    i += 1

0
The car, https://www.carmax.com/car/20924008, failed to load properly on loop iteration 7; skipping to next iteration.
10
The car, https://www.carmax.com/car/20931728, failed to load properly on loop iteration 12; skipping to next iteration.
The car, https://www.carmax.com/car/21162344, failed to load properly on loop iteration 12; skipping to next iteration.
20
30
40
The car, https://www.carmax.com/car/20883165, failed to load properly on loop iteration 42; skipping to next iteration.
The car, https://www.carmax.com/car/21162472, failed to load properly on loop iteration 47; skipping to next iteration.
50
60
70
The car, https://www.carmax.com/car/21313750, failed to load properly on loop iteration 71; skipping to next iteration.
80
The car, https://www.carmax.com/car/20929704, failed to load properly on loop iteration 85; skipping to next iteration.
90
100
The car, https://www.carmax.com/car/21020495, failed to load properly on loop iteration 107; skipping to next iteration.
The car

KeyboardInterrupt: 

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Curb Weight                 91 non-null     object
 1   Drive Type                  253 non-null    object
 2   Driver Head Room            137 non-null    object
 3   Driver Leg Room             137 non-null    object
 4   Engine Type                 253 non-null    object
 5   Exterior/Interior Color     253 non-null    object
 6   Front Tire Size (standard)  137 non-null    object
 7   Fuel Capacity               137 non-null    object
 8   Horsepower                  137 non-null    object
 9   MPG                         179 non-null    object
 10  Number of Keys              253 non-null    object
 11  Passenger Capacity          138 non-null    object
 12  Prior Use                   253 non-null    object
 13  Stock #                     253 non-null    object

In [21]:
df.to_csv('car_list_pickup.csv')

In [ ]:

for key in cat_lst:
    cat = car.find('span', class_ = key).get('class')[0]
    val = car.find('span', class_ = key).text
    
    car_dict[cat] = val


In [ ]:
car_dict

In [ ]:
spfc_car = soup.find('article', class_ = 'car-tile').find('a').get('href')
car_link = 'https://www.carmax.com' + spfc_car

car_link

In [ ]:
driver_car = webdriver.Firefox()
driver_car.get(car_link)

In [ ]:
feat_spec = driver_car.find_element_by_xpath('//div[contains(@class, "information-drawers")]/div[contains(@class, "drawer")]/div[contains(@class, "drawer-header")]/span[contains(@class, "svg-icon-wrapper")]')
driver_car.execute_script("arguments[0].scrollIntoView(true);", feat_spec)

In [ ]:
feat_spec.click()

In [ ]:
soup_car = BeautifulSoup(driver_car.page_source, 'html.parser')

In [ ]:
specs = soup_car.find('div', id = "specifications-overview")

In [ ]:
spec_dict = {}
for spec in specs.find_all('div', class_ = "specification"):
    print(spec.find_all('p')[1].text)
    ps = spec.find_all('p')
    spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text
    
spec_dict

In [ ]:
perf_specs = soup_car.find_all('div', class_ = "specifications-overview")[1]

In [ ]:
for spec in perf_specs.find_all('div', class_ = "specification"):
    print(spec.find_all('p')[1].text)
    ps = spec.find_all('p')
    spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text

In [ ]:
meas_specs = soup_car.find_all('div', class_ = "specifications-overview")[2]

In [ ]:
for spec in meas_specs.find_all('div', class_ = "specification"):
    print(spec.find_all('p')[1].text)
    ps = spec.find_all('p')
    spec_dict[spec.find_all('p')[0].text] = spec.find_all('p')[1].text

In [ ]:
spec_dict

In [ ]:
driver_car.close()

In [ ]:
car_dict.update(spec_dict)
car_dict